In [1]:
# Pandas and numpy for data manipulation
import pandas as pd
import numpy as np
import os
import pandas_profiling
pd.set_option('display.max_columns', 2000)

import requests
import json
from pandas.io.json import json_normalize #package for flattening json in pandas df

# Matplotlib and seaborn for visualization
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as plt
from pylab import rcParams
import seaborn as sb

%matplotlib inline 

# NLP modules for manipulating text data in app (nltk.download)
import re
import string
import nltk

# Scipy for statistics
import scipy

# Sklearn for modeling
import sklearn.preprocessing as preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer, LabelEncoder, OneHotEncoder

In [2]:
mainDir = 'C:/Users/Reuven/Documents/Python Scripts/Iron Source'
curDir = os.getcwd()
print(curDir)

C:\Users\Reuven\Documents\Python Scripts\Iron Source


## Helper Functions

### Grouping Features by their types for furthet transformation

In [3]:
TXT_tags = ['description', 'advisories', 'genres', 'genreIds', 'languageCodesISO2A']

CAT_tags = ['artistId', 'artistName', 'primaryGenreId', 'primaryGenreName',
            'contentAdvisoryRating', 'trackContentRating', 'currency', 'kind']

NUM_tags = ['averageUserRating', 'averageUserRatingForCurrentVersion']
            
AVG_tags = ['userRatingCount', 'userRatingCountForCurrentVersion'] # use for better ranking based on user rating

USE_tags = TXT_tags + CAT_tags + NUM_tags + ['trackId']

ALL_tags = ['description', 'genres', 'advisories',
            'isGameCenterEnabled', 'kind', 'features',
            'supportedDevices', 'averageUserRatingForCurrentVersion', 'trackCensoredName', 'languageCodesISO2A',
            'fileSizeBytes', 'contentAdvisoryRating', 'userRatingCountForCurrentVersion', 'trackViewUrl',
            'trackContentRating', 'currentVersionReleaseDate', 'trackId', 'trackName', 'currency', 'wrapperType',
            'version', 'artistId', 'artistName', 'price', 'bundleId', 'formattedPrice',
            'releaseNotes', 'isVppDeviceBasedLicensingEnabled', 'primaryGenreName', 'primaryGenreId',
            'sellerName', 'releaseDate', 'minimumOsVersion', 'genreIds', 'averageUserRating', 'userRatingCount']

### AppRecommend  API

In [18]:
def AppRecommend(currentAppID, eligibleApps=[]):
    
    # check input app list validity
    appList = AppCreateCombinedList(currentAppID, eligibleApps)
    
    # Create http request from combined list
    app_request = AppCreateRequestStr(appList)
    
    # fetch all apps data into json format
    response = AppRequestRemoteData(app_request)
   
    # parse json response info DataFrame
    dfResponse = LoadResponseToDataframe(response)
    trainData = dfResponse[USE_tags].copy()
    trainData.set_index('trackId', inplace=True)
    
    # check if currentAppID returned with data
    try:
        trainData.loc[currentAppID]
    except:
        print("ERROR: inavlid/missing App ID:", currentAppID)
        return
    
    # get featured dataset in a new Dataframe
    print("INFO: perform feature engineering...")
    X_features = AppFeatureEngineering(trainData)
    
    # compute pairwise cosine similarity (among other metrics)
    score_df = get_cosine_similarity(X_features, currentAppID)
    
    # save ranking into file
    df = save_recommendation_file(score_df, currentAppID)
    
    return df

### Helper function for manipulating json files request and response

In [19]:
# create combined list of current app and eligble apps
def AppCreateCombinedList(currentAppID, eligibleApps):
    try:
        appList = eligibleApps
        len(eligibleApps)
    except TypeError:
        appList = [eligibleApps]
    appList.append(currentAppID)
    return list(set(appList))

def AppCreateRequestStr(appList):
    app_str = ','.join([str(x) for x in appList])
    app_request = 'http://itunes.apple.com/lookup?id=' + app_str
    return app_request
    

# request and download all apps for the first time
def AppRequestRemoteData(request):
    try:
        response = json.loads(requests.get(request).text)
    except:
        print("ERROR: occured while fetching remote App data!")
    return response

    
# create http request, get the app data, save into file and return json string
def RequestAndSaveAppFile(appList):
    try:
        len(appList)
    except TypeError:
        appList = [appList]
    for appId in appList:
        appRequest = 'http://itunes.apple.com/lookup?id=' + str(appId)
        saveToFile = mainDir +'/app_' + str(appId) + '.txt'
        data = json.loads(requests.get(appRequest).text)
        with open(saveToFile, "w") as fileWriter:
            json.dump(data, fileWriter)
    return data


# load json file for single app
def LoadAppInfoFromFile(fileName):
    with open(fileName, 'r') as fileReader:
        data = json.load(fileReader)
    return data


# get json string for aingle or multiple apps and create DataFrame
def LoadResponseToDataframe(response):
    data = response['results'][0]
    origDf = json_normalize(response['results'])
    return origDf

### Customized Tokenization Functions

In [20]:
# Functions to remove punctuation, tokenize, remove stopwords and Stem/Lemm
stopwords = nltk.corpus.stopwords.words('english')

# cleaning, tokenizing and stemming text features
def CleanAndStemmText(text, ngram=1):
    ps = nltk.PorterStemmer()
    text = re.sub(',/', ' ', text)
    text = "".join([word.lower() for word in text if word not in string.punctuation + string.digits])
    text = str(text).replace(',', ' ').replace('/', ' ')
    tokens = re.split('\W', text)
    if ngram > 1:
        text = " ".join([ps.stem(word) for word in tokens if word not in stopwords])
    else:
        text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text


# cleaning, tokenizing and lemmatizing text features
def CleanAndLemmText(text, ngram=1):
    wn = nltk.WordNetLemmatizer()
    text = str(text).replace(',', ' ').replace('/', ' ')
    text = "".join([word.lower() for word in text if word not in string.punctuation + string.digits])
    tokens = re.split('\W', text)
    if ngram > 1:
        text = " ".join([wn.lemmatize(word) for word in tokens if word not in stopwords])
    else:
        text = [wn.lemmatize(word) for word in tokens if word not in stopwords]
    return text


# splitting, tokenizing and lemmatizing text features
def TokenizeAndLemmatizeText(text):
    wn = nltk.WordNetLemmatizer()
    if type(text) is list:
        text = " ".join(text) # convert array to str
    text = str(text).replace(',', ' ').replace('/', ' ')
    text = " ".join(re.findall("[\w]{2,}", text))
    text = "".join([word.lower() for word in text if word not in string.punctuation + string.digits])
    tokens = nltk.word_tokenize(text)
    text = [wn.lemmatize(word) for word in tokens if word not in stopwords]
    return text

### Helper Classes for FeatureUnion with DataFarme (would help peserving feature names)

In [21]:
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.externals.joblib import Parallel, delayed, Memory
from sklearn.pipeline import FeatureUnion, _fit_transform_one, _transform_one
from scipy import sparse

# Apply function on column(s)
class PandasTransformer(TransformerMixin, BaseEstimator):
    def __init__(self, fn):
        self.fn = fn

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None, copy=None):
        return self.fn(X)
    
# Make sure the returned Object is either sparse matrix or Dataframe
class PandasFeatureUnion(FeatureUnion):
    def fit_transform(self, X, y=None, **fit_params):
        self._validate_transformers()
        result = Parallel(n_jobs=self.n_jobs)(
            delayed(_fit_transform_one)(trans, weight, X, y, **fit_params)
            for name, trans, weight in self._iter())

        if not result:
            # All transformers are None
            return np.zeros((X.shape[0], 0))
        Xs, transformers = zip(*result)
        self._update_transformer_list(transformers)
        if any(sparse.issparse(f) for f in Xs):
            Xs = sparse.hstack(Xs).tocsr()
        else:
            Xs = self.merge_dataframes_by_column(Xs)
        return Xs

    def merge_dataframes_by_column(self, Xs):
        return pd.concat(Xs, axis="columns", copy=False)

    def transform(self, X):
        Xs = Parallel(n_jobs=self.n_jobs)(
            delayed(_transform_one)(trans, weight, X)
            for name, trans, weight in self._iter())
        if not Xs:
            # All transformers are None
            return np.zeros((X.shape[0], 0))
        if any(sparse.issparse(f) for f in Xs):
            Xs = sparse.hstack(Xs).tocsr()
        else:
            Xs = self.merge_dataframes_by_column(Xs)
        return Xs

## Feature Engineering and Data Vectorization

In [22]:
def AppFeatureEngineering(trainData):
    
    # Notes (for improvement etc.):
    # 1. Dimensionality reduction on text attributes
    # 2. Combination of metrics on different feature types, e.g. manhatthan dist for categorical/One-Hot
    # 3. Different Wieights for different types of features
    # 4. Clustering of wider/entire space
    # 5. Caching the results for future fast online retrieval
    
    # define tokenizer for text attributes - TEX_tags
    spec_tokenizer = lambda x: TokenizeAndLemmatizeText(x)
    
    # In this case, TEXT features can be an array of arrays of strings per document and not just strings.
    # In order to be able to run Fit, we exclude the prepocessor step (because there are no raw documents):
    
    tfidf_vect = TfidfVectorizer(input='content', preprocessor=lambda x: x, tokenizer=spec_tokenizer,
                                 analyzer='word', stop_words='english', ngram_range=(1,2))
    
    # tfidf for list/arrays that do not need special tokenizing, e.g. 'genreIds'
    tfidf_list = TfidfVectorizer(preprocessor=lambda x: x, tokenizer=lambda x: x, ngram_range=(1,1))
    
    # build feature union pipileline to transform and combine the different feature types
    union = PandasFeatureUnion(
    transformer_list = [
        ('description', Pipeline([
            ('select', PandasTransformer(lambda x: x['description'])),
            ('tfidf',  tfidf_vect),
            ('dframe', PandasTransformer(lambda x: pd.DataFrame(x.toarray(),columns=tfidf_vect.get_feature_names())))
        ])),
            
        ('advisories', Pipeline([
            ('select', PandasTransformer(lambda x: x['advisories'])),
            ('tfidf',  tfidf_vect),
            ('dframe', PandasTransformer(lambda x: pd.DataFrame(x.toarray(),columns=tfidf_vect.get_feature_names())))
        ])),
            
        ('genreIds',   Pipeline([
            ('select', PandasTransformer(lambda x: x['genreIds'])),
            ('tfidf',  tfidf_list),
            ('dframe', PandasTransformer(lambda x: pd.DataFrame(x.toarray(),columns=tfidf_list.get_feature_names())))
        ])),
    
        ('dummies', Pipeline([
            ('select', PandasTransformer(lambda x: pd.get_dummies(x, columns=CAT_tags, drop_first=False))),
            ('index', PandasTransformer(lambda x: x.reset_index()))
        ]))
    ])
    X_union = union.fit_transform(trainData)

    X_union = X_union.drop(axis=1, columns=TXT_tags)

    print(X_union.shape)
    print(type(X_union))

    scaler_minmax = preprocessing.MinMaxScaler(copy=False)
    X_union[NUM_tags] = scaler_minmax.fit_transform(X_union[NUM_tags].values)
    
    X_union.set_index('trackId', inplace=True)
    
    return X_union

### Pairwise Metrics Computation

In [23]:
from sklearn.metrics.pairwise import cosine_similarity

# cosine metric better for orientation similarity
def get_cosine_similarity(X_features, app_id):
    print("INFO: compute Cosine Simliarity for AppId: %d" % app_id)
    y_vec = X_features.loc[app_id]
    sc_cosine = cosine_similarity(X_features)
    df_cosine = pd.DataFrame(sc_cosine, index=X_features.index, columns=X_features.index)
    df_cosine = df_cosine.loc[app_id].sort_values(ascending=False)
    
    return df_cosine

# save scoring into csv file
def save_recommendation_file(score_list, app_id):
    file_name = str(app_id) +'_recommend.csv'
    df = pd.DataFrame(score_list)
    df.rename_axis('AppID', inplace=True)
    df.set_axis(['Score'], axis=1, inplace=True)
    df = df.drop([app_id])
    try:
        df.to_csv(file_name)
        print("INFO: scoring saved into: %s" %file_name)
    except:
        print("ERROR: failed saving file: %s" %file_name)
    return df

## Main and Testing

In [24]:
app_list = [429775439, 506627515, 504720040, 447553564, 440045374, 512939461]
AppRecommend(9999, app_list)
#apps = AppCreateCombinedList(9999, app_list)

ERROR: inavlid/missing App ID: 9999


In [54]:
apps

[512939461, 504720040, 429775439, 9999, 506627515, 447553564, 440045374]

## Request App data and Save to File

In [5]:
app_list = [429775439, 506627515, 504720040, 447553564, 440045374, 512939461]
filename = 'C:/Users/Reuven/Documents/Python Scripts/Iron Source/429775439.txt'
request_single_app = 'http://itunes.apple.com/lookup?id=429775439'
app_str = ','.join([str(x) for x in app_list])  # list comprehension
request_multi_app = 'http://itunes.apple.com/lookup?id=' + app_str

In [6]:
# given list of apps create html request and fetch & save data
appinfo = RequestAndSaveAppFile(app_list)
response = json.loads(requests.get(request_multi_app).text)

In [8]:
# Create a DataFrame containg all apps with selected columns
df = LoadResponseToDataframe(response)
trainData = df[USE_tags].copy()
trainData.set_index('trackId', inplace=True)
trainData.loc[512939461]

description                           Presented by Kiloo Games and Sybo Games.\n\nDA...
advisories                                [Infrequent/Mild Cartoon or Fantasy Violence]
genres                                           [Games, Action, Arcade, Entertainment]
genreIds                                                       [6014, 7001, 7003, 6016]
languageCodesISO2A                                                         [EN, PT, ES]
artistId                                                                      330629809
artistName                                                                        Kiloo
primaryGenreId                                                                     6014
primaryGenreName                                                                  Games
contentAdvisoryRating                                                                9+
trackContentRating                                                                   9+
currency                        

In [9]:
print("Missing values:\n%s" % trainData.isnull().sum().sum())

Missing values:
0


## Vectorizing features using CountVectorizer

In [10]:
X_description = trainData['description'].apply(lambda x: CleanAndLemmText(x, ngram=2))
ngram_vect = CountVectorizer(ngram_range=(1,2))
X_ngram = ngram_vect.fit_transform(X_description)
print(X_ngram.shape)
X_ngram_df = pd.DataFrame(X_ngram.toarray(), columns=ngram_vect.get_feature_names())
X_ngram_df

(6, 1532)


,absurdly,absurdly fun,accessible,accessible certain,accompanied,accompanied original,achieve,achieve dragonly,achieve ultimate,achievement,achievement allnew,acrobatics,acrobatics challenge,activity,activity special,addictive,addictive series,addition,addition getting,adventure,adventure game,adventure win,age,age privacy,air,air plus,album,album many,album premium,album win,allnew,allnew collection,allnew game,allnew sloto,allowed,allowed download,along,along way,also,also get,also purchased,also required,also smooth,always,always tap,amazing,amazing exclusive,amazing reward,amusement,amusement purpose,animal,animal make,animal super,animation,animation accompanied,anything,anything ever,app,app hd,app purchase,app slotomania,app store,app time,apply,apply home,arcade,arcade twist,arrived,arrived incredible,artwork,artwork phenomenal,associated,associated app,awaits,awaits dragonvale,awardwinning,awardwinning composer,away,away head,awesome,awesome casino,back,back linked,backflip,backflip studio,ball,ball button,bank,bank gift,barry,barry much,based,based entirely,beautiful,beautiful floating,become,become part,become slot,become winner,beef,beef trophy,begin,begin discover,beginning,beginning invited,benefit,benefit like,best,best casino,best farm,better,better fun,big,big casino,big payouts,big premiere,bingo,bingo earn,bingo every,bingo franchise,bingo game,bingo madness,bingo odds,bingo play,bingo possible,bingo rule,bingo rush,bingo travel,bingo view,bingorush,bingorush feel,bingorush follow,bingorush problem,blackout,blackout bingo,blast,blast awesome,bonus,bonus break,bonus get,booster,booster hourly,box,box office,brag,brag big,brag facebook,branch,branch farm,break,break daily,break piggy,breath,breath slotomania,breed,breed impress,breed raise,breeding,breeding visit,brought,brought backflip,browse,browse dragonarium,buffalostudioscom,buffalostudioscom privacypolicy,build,build decorate,build farm,build town,button,button bingo,called,called innovative,candy,candy control,card,card brag,card chance,card collect,card collection,card earn,card even,card friend,card increase,card purchase,card trade,cash,cash back,cash become,casino,casino app,casino bonus,casino everyone,casino feature,casino fun,casino game,casino gaming,casino slot,cast,cast lure,certain,certain territory,certainly,certainly comical,challenge,challenge help,chance,chance win,chance winning,channel,channel service,chase,chase universal,check,check facebook,choice,choice made,cinematic,cinematic mobile,city,city collection,classic,classic bingo,classic game,clear,clear everyone,clear missing,clock,clock collect,club,club become,club enjoy,coin,coin become,coin booster,coin daily,coin exciting,coin get,coin powerups,collect,collect card,collect complete,collect many,collect sloto,collected,collected yet,collecting,collecting game,collection,collection achievement,collection allnew,collection arrived,collection brag,collection item,colorful,colorful vivid,comedy,comedy special,comical,comical animal,community,community million,community spin,compatible,compatible iphone,compete,compete friend,complete,complete album,complete city,composer,composer play,computer,computer connected,connect,connect friend,connected,connected tv,connection,connection also,connection play,contact,contact game,content,content regular,contest,contest win,control,control also,control quickly,cool,cool crew,cooperative,cooperative breeding,country,country learn,create,create ultimate,credit,credit coin,crew,crew colorful,crop,crop fresh,crop grow,crop never,crop trade,cultural,cultural epicenter,customize,customize farm,cute,cute nothing,daily,daily collect,daily routine,daring,daring chase,dash,dash fast,day,day allowed,day free,day love,day popular,day presentation,decorate,decorate beautiful,decoration,decoration activity,design,design thought,despicable,despicable wolf,destiny,destiny awaits,detail,detail certainly,device,device hbo,device setting,die,die even,different,di

## Vectorizing features using TfidfVectorizer

In [11]:
# In this case, TEXT features can be an array of arrays of strings per document and not just strings.
# In order to be able to run Fit, we exclude the prepocessor step (because there are no raw documents):

spec_tokenizer = lambda x: TokenizeAndLemmatizeText(x)
tfidf_vect = TfidfVectorizer(input='content', preprocessor=lambda x: x, tokenizer=spec_tokenizer,
                             analyzer='word', stop_words='english', ngram_range=(1,2))

X_tfidf = tfidf_vect.fit_transform(trainData['description'])

print(X_tfidf.shape)
X_tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vect.get_feature_names())
X_tfidf_df

(6, 1444)


absurdly  absurdly fun  accessible  accessible certain  accompanied  \
0  0.000000      0.000000    0.071152            0.071152     0.000000   
1  0.000000      0.000000    0.000000            0.000000     0.000000   
2  0.000000      0.000000    0.000000            0.000000     0.000000   
3  0.000000      0.000000    0.000000            0.000000     0.000000   
4  0.053393      0.053393    0.000000            0.000000     0.053393   
5  0.000000      0.000000    0.000000            0.000000     0.000000   

   accompanied original   achieve  achieve dragonly  achieve ultimate  \
0              0.000000  0.000000          0.000000          0.000000   
1              0.000000  0.000000          0.000000          0.000000   
2              0.000000  0.031681          0.000000          0.038635   
3              0.000000  0.000000          0.000000          0.000000   
4              0.053393  0.043783          0.053393          0.000000   
5              0.000000  0.000000          0.000000          0.000000   

   achievement  achievement new  acrobatics  acrobatics challenge  activity  \
0     0.000000         0.000000    0.000000              0.000000  0.000000   
1     0.000000         0.000000    0.000000              0.000000  0.000000   
2     0.038635         0.038635    0.000000              0.000000  0.000000   
3     0.000000         0.000000    0.000000              0.000000  0.000000   
4     0.000000         0.000000    0.000000              0.000000  0.053393   
5     0.000000         0.000000    0.089712              0.089712  0.000000   

   activity special  addictive  addictive series  addition  addition getting  \
0          0.000000   0.071152          0.071152  0.071152          0.071152   
1          0.000000   0.000000          0.000000  0.000000          0.000000   
2          0.000000   0.000000          0.000000  0.000000          0.000000   
3          0.000000   0.000000          0.000000  0.000000          0.000000   
4          0.053393   0.000000          0.000000  0.000000          0.000000   
5          0.000000   0.000000          0.000000  0.000000          0.000000   

   adventure  adventure game  adventure win       age  age privacy       air  \
0   0.000000        0.000000       0.000000  0.000000     0.000000  0.071152   
1   0.000000        0.000000       0.000000  0.036945     0.036945  0.000000   
2   0.000000        0.000000       0.000000  0.000000     0.000000  0.000000   
3   0.044344        0.022172       0.022172  0.000000     0.000000  0.000000   
4   0.000000        0.000000       0.000000  0.000000     0.000000  0.000000   
5   0.000000        0.000000       0.000000  0.000000     0.000000  0.000000   

   air plus     album  album premium  album way  album win   allowed  \
0  0.071152  0.000000       0.000000   0.000000   0.000000  0.000000   
1  0.000000  0.000000       0.000000   0.000000   0.000000  0.036945   
2  0.000000  0.000000       0.000000   0.000000   0.000000  0.000000   
3  0.000000  0.066517       0.022172   0.022172   0.022172  0.000000   
4  0.000000  0.000000       0.000000   0.000000   0.000000  0.000000   
5  0.000000  0.000000       0.000000   0.000000   0.000000  0.000000   

   allowed download   amazing  amazing exclusive  amazing reward  amusement  \
0          0.000000  0.000000           0.000000        0.000000   0.000000   
1          0.036945  0.000000           0.000000        0.000000   0.000000   
2          0.000000  0.000000           0.000000        0.000000   0.000000   
3          0.000000  0.044344           0.022172        0.022172   0.022172   
4          0.000000  0.000000           0.000000        0.000000   0.000000   
5          0.000000  0.000000           0.000000        0.000000   0.000000   

   amusement purpose   animal  animal make  animal super  animation  \
0           0.000000  0.00000     0.000000      0.000000   0.000000   
1           0.000000  0.07389     0.036945      0.036945   0.000000   
2           0.000000  0.

In [12]:
tfidf_vect = TfidfVectorizer(preprocessor=lambda x: x, tokenizer=spec_tokenizer,
                             stop_words='english', ngram_range=(1,2))
X_tfidf = tfidf_vect.fit_transform(trainData['genres'])
print(X_tfidf.shape)
X_tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vect.get_feature_names())
X_tfidf_df

# tfidf for list that do not need tokenizing, e.g. 'genreIds'
tfidf_list = TfidfVectorizer(preprocessor=lambda x: x, tokenizer=lambda x: x,
                             ngram_range=(1,1))
X_tfidf_list = tfidf_list.fit_transform(trainData['genreIds'])
print(X_tfidf_list.shape)
X_tfidf_list_df = pd.DataFrame(X_tfidf_list.toarray(), columns=tfidf_list.get_feature_names())
X_tfidf_list_df

(6, 21)
(6, 9)


,6012,6014,6016,7001,7003,7005,7006,7009,7015
0,0.889996,0.000000,0.455969,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.374670,0.374670,0.000000,0.000000,0.000000,0.000000,0.599685,0.599685
2,0.000000,0.374670,0.374670,0.000000,0.599685,0.000000,0.599685,0.000000,0.000000
3,0.000000,0.345616,0.345616,0.000000,0.000000,0.674602,0.553183,0.000000,0.000000
4,0.000000,0.404106,0.000000,0.000000,0.000000,0.000000,0.000000,0.646799,0.646799
5,0.000000,0.345616,0.345616,0.674602,0.553183,0.000000,0.000000,0.000000,0.000000


## Use FeatureUnion to combine the different features types

In [53]:
union = PandasFeatureUnion(
    transformer_list = [
        ('description', Pipeline([
            ('select', PandasTransformer(lambda x: x['description'])),
            ('tfidf',  tfidf_vect),
            ('dframe', PandasTransformer(lambda x: pd.DataFrame(x.toarray(),columns=tfidf_vect.get_feature_names())))
        ])),
            
        ('advisories', Pipeline([
            ('select', PandasTransformer(lambda x: x['advisories'])),
            ('tfidf',  tfidf_vect),
            ('dframe', PandasTransformer(lambda x: pd.DataFrame(x.toarray(),columns=tfidf_vect.get_feature_names())))
        ])),
            
        ('genreIds',   Pipeline([
            ('select', PandasTransformer(lambda x: x['genreIds'])),
            ('tfidf',  tfidf_list),
            ('dframe', PandasTransformer(lambda x: pd.DataFrame(x.toarray(),columns=tfidf_list.get_feature_names())))
        ])),
    
        ('dummies', Pipeline([
            ('select', PandasTransformer(lambda x: pd.get_dummies(x, columns=CAT_tags, drop_first=False))),
            ('index', PandasTransformer(lambda x: x.reset_index()))
        ]))
])
X_union = union.fit_transform(trainData)

X_union = X_union.drop(axis=1, columns=TXT_tags)

print(X_union.shape)
print(type(X_union))

scaler_minmax = preprocessing.MinMaxScaler(copy=False)
X_union[NUM_tags] = scaler_minmax.fit_transform(X_union[NUM_tags].values)

(6, 1543)
<class 'pandas.core.frame.DataFrame'>


## Pairwise Metrics Computation

In [54]:
# Notes:
# 1. Dimensionality reduction on text attributes
# 2. Combination of metrics on different feature types, e.g. manhatthan dist for categorical/One-Hot
# 3. Different Wieights for different types of features
# 4. Clustering of wider/entire space
# 5. Caching the results for future fast online retrieval

X_union.set_index('trackId', inplace=True)
X_union

absurdly  absurdly fun  accessible  accessible certain  \
trackId                                                             
429775439  0.000000      0.000000    0.071152            0.071152   
506627515  0.000000      0.000000    0.000000            0.000000   
504720040  0.000000      0.000000    0.000000            0.000000   
447553564  0.000000      0.000000    0.000000            0.000000   
440045374  0.053393      0.053393    0.000000            0.000000   
512939461  0.000000      0.000000    0.000000            0.000000   

           accompanied  accompanied original   achieve  achieve dragonly  \
trackId                                                                    
429775439     0.000000              0.000000  0.000000          0.000000   
506627515     0.000000              0.000000  0.000000          0.000000   
504720040     0.000000              0.000000  0.031681          0.000000   
447553564     0.000000              0.000000  0.000000          0.000000   
440045374     0.053393              0.053393  0.043783          0.053393   
512939461     0.000000              0.000000  0.000000          0.000000   

           achieve ultimate  achievement  achievement new  acrobatics  \
trackId                                                                 
429775439          0.000000     0.000000         0.000000    0.000000   
506627515          0.000000     0.000000         0.000000    0.000000   
504720040          0.038635     0.038635         0.038635    0.000000   
447553564          0.000000     0.000000         0.000000    0.000000   
440045374          0.000000     0.000000         0.000000    0.000000   
512939461          0.000000     0.000000         0.000000    0.089712   

           acrobatics challenge  activity  activity special  addictive  \
trackId                                                                  
429775439              0.000000  0.000000          0.000000   0.071152   
506627515              0.000000  0.000000          0.000000   0.000000   
504720040              0.000000  0.000000          0.000000   0.000000   
447553564              0.000000  0.000000          0.000000   0.000000   
440045374              0.000000  0.053393          0.053393   0.000000   
512939461              0.089712  0.000000          0.000000   0.000000   

           addictive series  addition  addition getting  adventure  \
trackId                                                              
429775439          0.071152  0.071152          0.071152   0.000000   
506627515          0.000000  0.000000          0.000000   0.000000   
504720040          0.000000  0.000000          0.000000   0.000000   
447553564          0.000000  0.000000          0.000000   0.044344   
440045374          0.000000  0.000000          0.000000   0.000000   
512939461          0.000000  0.000000          0.000000   0.000000   

           adventure game  adventure win       age  age privacy       air  \
trackId                                                                     
429775439        0.000000       0.000000  0.000000     0.000000  0.071152   
506627515        0.000000       0.000000  0.036945     0.036945  0.000000   
504720040        0.000000       0.000000  0.000000     0.000000  0.000000   
447553564        0.022172       0.022172  0.000000     0.000000  0.000000   
440045374        0.000000       0.000000  0.000000     0.000000  0.000000   
512939461        0.000000       0.000000  0.000000     0.000000  0.000000   

           air plus     album  album premium  album way  album win   allowed  \
trackId                                                                        
429775439  0.071152  0.000000       0.000000   0.000000   0.000000  0.000000   
506627515  0.000000  0.000000       0.000000   0.000000   0.000000  0.036945   
504720040  0.000000  0.000000       0.000000   0.000000   0.000000  0.000000   
447553564  0.000000  0.066517       0.022172   0.022172   0.022172  0.000000   
440045374  0.

In [63]:
# cosine metric better for orientation similarity
from sklearn.metrics.pairwise import cosine_similarity
app_id = 429775439
y_vec = X_union.loc[app_id]
sc_cosine = cosine_similarity(X_union)
df_cosine = pd.DataFrame(sc_cosine, index=X_union.index, columns=X_union.index)
df_cosine.loc[app_id].sort_values(ascending=False)
df_cosine

trackId,429775439,506627515,504720040,447553564,440045374,512939461
trackId,,,,,,
429775439,1.000000,0.195596,0.232590,0.223392,0.175593,0.213658
506627515,0.195596,1.000000,0.477102,0.494060,0.749007,0.503409
504720040,0.232590,0.477102,1.000000,0.722069,0.462616,0.479269
447553564,0.223392,0.494060,0.722069,1.000000,0.483564,0.470780
440045374,0.175593,0.749007,0.462616,0.483564,1.000000,0.492491
512939461,0.213658,0.503409,0.479269,0.470780,0.492491,1.000000


In [66]:
# sigmoid kernel
from sklearn.metrics.pairwise import sigmoid_kernel
sc_sigmod = sigmoid_kernel(X_union)
df_sigmod = pd.DataFrame(sc_sigmod, index=X_union.index, columns=X_union.index)
df_sigmod.loc[app_id].sort_values(ascending=False)
df_sigmod

trackId,429775439,506627515,504720040,447553564,440045374,512939461
trackId,,,,,,
429775439,0.764574,0.762206,0.762319,0.762308,0.762143,0.762289
506627515,0.762206,0.764843,0.763144,0.763241,0.764031,0.763301
504720040,0.762319,0.763144,0.764822,0.763990,0.763097,0.763214
447553564,0.762308,0.763241,0.763990,0.764994,0.763206,0.763228
440045374,0.762143,0.764031,0.763097,0.763206,0.764843,0.763264
512939461,0.762289,0.763301,0.763214,0.763228,0.763264,0.765112


In [68]:
# RBF kernel
from sklearn.metrics.pairwise import rbf_kernel
sc_rbf = rbf_kernel(X_union)
df_rbf = pd.DataFrame(sc_rbf, index=X_union.index, columns=X_union.index)
df_rbf.loc[app_id].sort_values(ascending=False)
df_rbf

trackId,429775439,506627515,504720040,447553564,440045374,512939461
trackId,,,,,,
429775439,1.000000,0.988071,0.988654,0.988196,0.987776,0.987824
506627515,0.988071,1.000000,0.991921,0.991971,0.996101,0.991975
504720040,0.988654,0.991921,1.000000,0.995593,0.991698,0.991612
447553564,0.988196,0.991971,0.995593,1.000000,0.991805,0.991264
440045374,0.987776,0.996101,0.991698,0.991805,1.000000,0.991799
512939461,0.987824,0.991975,0.991612,0.991264,0.991799,1.000000


In [61]:
# let's try euclidian which may be good for sparse data
from sklearn.metrics.pairwise import euclidean_distances
sc_euclid = euclidean_distances(X_union)
df_euclid = pd.DataFrame(sc_euclid, index=X_union.index, columns=X_union.index)
df_euclid.loc[app_id].sort_values(ascending=False)
df_euclid

trackId,429775439,506627515,504720040,447553564,440045374,512939461
trackId,,,,,,
429775439,0.000000,4.301807,4.194672,4.279068,4.354903,4.346266
506627515,4.301807,0.000000,3.536818,3.525669,2.454350,3.524891
504720040,4.194672,3.536818,0.000000,2.609825,3.585474,3.604039
447553564,4.279068,3.525669,2.609825,0.000000,3.562034,3.678309
440045374,4.354903,2.454350,3.585474,3.562034,0.000000,3.563365
512939461,4.346266,3.524891,3.604039,3.678309,3.563365,0.000000


In [64]:
# polynomial kernels: considers not only the similarity between vectors under the same dimension,
# but also across dimension, so this allows to account for feature interaction.
from sklearn.metrics.pairwise import polynomial_kernel
sc_polynm = polynomial_kernel(X_union)
df_polynm = pd.DataFrame(sc_polynm, index=X_union.index, columns=X_union.index)
df_polynm.loc[app_id].sort_values(ascending=False)
df_polynm

trackId,429775439,506627515,504720040,447553564,440045374,512939461
trackId,,,,,,
429775439,1.021554,1.004378,1.005191,1.005118,1.003930,1.004979
506627515,1.004378,1.023528,1.011144,1.011848,1.017589,1.012283
504720040,1.005191,1.011144,1.023375,1.017291,1.010804,1.011653
447553564,1.005118,1.011848,1.017291,1.024640,1.011596,1.011751
440045374,1.003930,1.017589,1.010804,1.011596,1.023528,1.012015
512939461,1.004979,1.012283,1.011653,1.011751,1.012015,1.025506
